### Какое количество песен будет достаточным?
### Нужны ли цепи Маркова?

In [3]:
import lyricsgenius
import json
import pandas as pd
import nltk
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
nltk.download('punkt')
from nltk.lm.preprocessing import flatten
from nltk.tokenize import word_tokenize
from nltk.lm import KneserNeyInterpolated as KNI
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import word_tokenize, sent_tokenize 
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Наталья\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [5]:
#Checking whether installed module works correctly, setting Eminem as the author
genius = lyricsgenius.Genius("jHJB8vqrbfVDIGM1KrETFx6s7EG9SGN3RGcmr63z7Gjgz8dYY4LizTL9rGupTAl1")
#at first used to sort by title, but it brings remixes of same song, so let us sort by popularity 
artist = genius.search_artist("Eminem", max_songs=100, sort="popularity")
genius.remove_section_headers = True
genius.verbose = False
print('')

Searching for songs by Eminem...

Song 1: "Rap God"
Song 2: "Killshot"
Song 3: "Lose Yourself"
Song 4: "The Monster"
Song 5: "Lucky You"
Song 6: "Godzilla"
Song 7: "The Ringer"
Song 8: "River"
Song 9: "Berzerk"
Song 10: "Venom"
Song 11: "Not Alike"
Song 12: "Fall"
Song 13: "Stan"
Song 14: "Without Me"
Song 15: "The Real Slim Shady"
Song 16: "Walk on Water"
Song 17: "Kamikaze"
Song 18: "’Till I Collapse"
Song 19: "Bad Guy"
Song 20: "Love the Way You Lie"
Song 21: "Mockingbird"
Song 22: "8 Mile: B-Rabbit vs Papa Doc"
Song 23: "Headlights"
Song 24: "Survival"
Song 25: "Not Afraid"
Song 26: "No Love"
Song 27: "Beautiful"
Song 28: "Greatest"
Song 29: "When I’m Gone"
Song 30: "Love Game"
Song 31: "My Name Is"
Song 32: "Legacy"
Song 33: "Cleanin’ Out My Closet"
Song 34: "The Way I Am"
Song 35: "Space Bound"
Song 36: "Like Toy Soldiers"
Song 37: "Guts Over Fear"
Song 38: "Superman"
Song 39: "Unaccommodating"
Song 40: "Sing for the Moment"
Song 41: "Stronger Than I Was"
Song 42: "Believe"
Song 

In [6]:
artist.save_lyrics()

Wrote `Lyrics_Eminem.json`


In [7]:
lyrics = 'Lyrics_Eminem.json'
with open(lyrics) as eminem_lyrics:
    messyvocab = json.load(eminem_lyrics)
dict_keys = messyvocab.keys()
print(dict_keys)

dict_keys(['alternate_names', 'api_path', 'description', 'facebook_name', 'followers_count', 'header_image_url', 'id', 'image_url', 'instagram_name', 'is_meme_verified', 'is_verified', 'name', 'translation_artist', 'twitter_name', 'url', 'current_user_metadata', 'iq', 'description_annotation', 'user', 'songs'])


In [8]:
vocab = messyvocab.get('songs')
lyrics_vocab = []
for i in range(0,len(vocab)):
    index = vocab[i]
    song = index.get('lyrics')
    lyrics_vocab.append(song)
lyricsdf = pd.DataFrame(lyrics_vocab, columns=['Song lyrics'])
lyricsdf.head(3)

,Song lyrics
0,"[Intro]\n""Look, I was gonna go easy on you not..."
1,"[Intro]\nYou sound like a bitch, bitch\nShut t..."
2,"[Intro]\nLook, if you had one shot or one oppo..."


In [9]:
lyricsdf = lyricsdf.dropna()
print(len(lyricsdf))

99


In [59]:
for i in range(0,len(lyricsdf)):
    stringlyric = lyricsdf.iloc[i,0]
    stringlyric.replace("\n\n\n\n\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n\n", "\n")
    stringlyric.replace("\n\n\n", "\n")
    stringlyric.replace("\n\n", "\n")
    stringlyric.replace("\\n", "\n")
    stringlyric.replace("\\", "\n")
    for x in ['intro', 'verse', 'chorus','!','"', "'", '’', '#','$','%','&','(',')','*','+',',','-','/',':',';','<','=','>','?','@','[','\\',']',
 '^','_','`','{','|','}','~', '1', '...']:   
        stringlyric = stringlyric.replace(x, '')
        lyricsdf.iloc[i,0] = stringlyric

In [93]:
from nltk.lm.models import LanguageModel
#copypaste from NLTK source documentation
class InterpolatedLanguageModel(LanguageModel):
    def __init__(self, smoothing_cls, order, **kwargs):
        assert issubclass(smoothing_cls, Smoothing)
        params = kwargs.pop("params", {})
        super().__init__(order, **kwargs)
        self.estimator = smoothing_cls(self.vocab, self.counts, **params)


    def unmasked_score(self, word, context=None):
        if not context:
                return self.estimator.unigram_score(word)
        if not self.counts[context]:
             return self.unmasked_score(word, context[1:])
        alpha, gamma = self.estimator.alpha_gamma(word, context)
        return alpha + gamma * self.unmasked_score(word, context[1:])

In [94]:
def _count_non_zero_vals(dictionary):
    return sum(1.0 for c in dictionary.values() if c > 0)

In [95]:
#this is KneserNey from NLTK documentation but with some amendments from https://github.com/nltk/nltk/pull/2363/commits/ce74e449dc9526e19596b1c4a9c510bbb35812cc 
from nltk.lm.models import Smoothing
class KneserNey(Smoothing):
    def __init__(self, vocabulary, counter, discount=0.1, **kwargs):
        super(KneserNey, self).__init__(vocabulary, counter, *kwargs)
        super().__init__(vocabulary, counter, **kwargs)
        self.discount = discount

    def unigram_score(self, word):
        return 1.0 / len(self.vocab)

    def alpha_gamma(self, word, context):
        prefix_counts = self.counts[context]
        prefix_total_ngrams = prefix_counts.N()
        alpha = max(prefix_counts[word] - self.discount, 0.0) / prefix_total_ngrams
        gamma = self.discount * _count_non_zero_vals(prefix_counts) / prefix_total_ngrams
        return alpha, gamma

In [96]:
#copypaste from NLTK source documentation
class KneserNeyInterpolated(InterpolatedLanguageModel):
    def __init__(self, order, discount=0.1, **kwargs):
        super().__init__(KneserNey, order, params={"discount": discount}, **kwargs)

In [60]:
lyricsdf.head(2)

,Song lyrics,lyr_token
0,\nlook i was gonna go easy on you not to hurt ...,"[look, i, was, gon, na, go, easy, on, you, not..."
1,\nyou sound like a bitch bitch\nshut the fuck ...,"[you, sound, like, a, bitch, bitch, shut, the,..."


In [61]:
lyricsdf['Song lyrics'] = lyricsdf['Song lyrics'].str.lower()
lyricsdf['lyr_token'] = lyricsdf.apply(lambda row: nltk.word_tokenize(row['Song lyrics']), axis=1)

In [83]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(lyricsdf, test_size=0.7, random_state=11)
trainset = train['lyr_token'].tolist()
testset = test['lyr_token'].tolist()

### Биграммы

In [106]:
train_data, padded_sents = padded_everygram_pipeline(2, trainset)
lm_bigrams = KneserNeyInterpolated(2, discount=0.1)
lm_bigrams.fit(train_data, padded_sents)
print(lm_bigrams.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 4118 items>


In [107]:
# Fully copypasted from https://www.kaggle.com/alvations/n-gram-language-model-with-nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [113]:
for i in range(5):
    print(generate_sent(lm_bigrams, 10, random_seed=i))

sues family fightin and just get the fuckin knife is
bye to take em not give out the bowl milk
wet pile gettin capped and slap you fuckin—oh and now
discriminated against picket signs for all aall aall aall aall
dinner plates at and back in whoo ladidadi got bent


### Perplexity, are you ok?

### Для юни-, би- и триграмм перплексия одинаковая :(

In [109]:
testset = test['lyr_token'].tolist()

In [110]:
test_grams = str(testset)
test_bigrams = ngrams(test_grams.split(), 2)

In [111]:
bigrams_test = []
for grams in test_bigrams:
    bigrams_test.append(grams)
bigrams_test = tuple(bigrams_test)

### Перплексия для биграмм 4118

In [112]:
lm_bigrams.perplexity(bigrams_test)

4117.999999956597

### Юниграммы

In [116]:
train_data, padded_sents = padded_everygram_pipeline(1, trainset)
lm_unigrams = KneserNeyInterpolated(1, discount=0.1)
lm_unigrams.fit(train_data, padded_sents)
print(lm_unigrams.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 4116 items>


In [126]:
test_grams = str(testset)
test_unigrams = ngrams(test_grams.split(), 1)

In [127]:
unigrams_test = []
for grams in test_unigrams:
    unigrams_test.append(grams)
unigrams_test = tuple(unigrams_test)
unigrams_test

(("[['you',",),
 ("'fuckin',",),
 ("'groupie',",),
 ("'pick',",),
 ("'up',",),
 ("'the',",),
 ("'goddamn',",),
 ("'phone',",),
 ("'bitch',",),
 ("'where',",),
 ("'the',",),
 ("'fuck',",),
 ("'were',",),
 ("'you',",),
 ("'tuesday',",),
 ("'with',",),
 ("'who',",),
 ("'you',",),
 ("'say',",),
 ("'i',",),
 ("'wasnt',",),
 ("'at',",),
 ("'the',",),
 ("'studio',",),
 ("'bitch',",),
 ("'whatd',",),
 ("'you',",),
 ("'do',",),
 ("'screw',",),
 ("'dre',",),
 ("'you',",),
 ("'went',",),
 ("'there',",),
 ("'lookin',",),
 ("'for',",),
 ("'me',",),
 ("'boo',",),
 ("'that',",),
 ("'excuse',",),
 ("'is',",),
 ("'too',",),
 ("'lame',",),
 ("'keep',",),
 ("'playin',",),
 ("'me',",),
 ("'youre',",),
 ("'gon',",),
 ("'na',",),
 ("'end',",),
 ("'up',",),
 ("'with',",),
 ("'a',",),
 ("'huge',",),
 ("'goose',",),
 ("'egg',",),
 ("'you',",),
 ("'fake',",),
 ("'lyin',",),
 ("'slut',",),
 ("'you',",),
 ("'never',",),
 ("'told',",),
 ("'me',",),
 ("'you',",),
 ("'knew',",),
 ("'drake',",),
 ("'and',",),
 ("'lup

### Перплексия для юниграмм 4116

In [128]:
lm_unigrams.perplexity(unigrams_test)

4116.000000006382

### Триграммы

In [129]:
train_data, padded_sents = padded_everygram_pipeline(3, trainset)
lm_trigrams = KneserNeyInterpolated(3, discount=0.1)
lm_trigrams.fit(train_data, padded_sents)
print(lm_trigrams.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 4118 items>


In [130]:
test_grams = str(testset)
test_trigrams = ngrams(test_grams.split(), 3)

In [131]:
trigrams_test = []
for grams in test_trigrams:
    trigrams_test.append(grams)
trigrams_test = tuple(trigrams_test)
trigrams_test

(("[['you',", "'fuckin',", "'groupie',"),
 ("'fuckin',", "'groupie',", "'pick',"),
 ("'groupie',", "'pick',", "'up',"),
 ("'pick',", "'up',", "'the',"),
 ("'up',", "'the',", "'goddamn',"),
 ("'the',", "'goddamn',", "'phone',"),
 ("'goddamn',", "'phone',", "'bitch',"),
 ("'phone',", "'bitch',", "'where',"),
 ("'bitch',", "'where',", "'the',"),
 ("'where',", "'the',", "'fuck',"),
 ("'the',", "'fuck',", "'were',"),
 ("'fuck',", "'were',", "'you',"),
 ("'were',", "'you',", "'tuesday',"),
 ("'you',", "'tuesday',", "'with',"),
 ("'tuesday',", "'with',", "'who',"),
 ("'with',", "'who',", "'you',"),
 ("'who',", "'you',", "'say',"),
 ("'you',", "'say',", "'i',"),
 ("'say',", "'i',", "'wasnt',"),
 ("'i',", "'wasnt',", "'at',"),
 ("'wasnt',", "'at',", "'the',"),
 ("'at',", "'the',", "'studio',"),
 ("'the',", "'studio',", "'bitch',"),
 ("'studio',", "'bitch',", "'whatd',"),
 ("'bitch',", "'whatd',", "'you',"),
 ("'whatd',", "'you',", "'do',"),
 ("'you',", "'do',", "'screw',"),
 ("'do',", "'screw',

### Перплексия для триграмм 4118

In [132]:
lm_trigrams.perplexity(trigrams_test)

4117.999999956597

Heavily based on:
    https://www.kaggle.com/alvations/n-gram-language-model-with-nltk and other sources because I am stupid as f.ck